In [1]:
import pandas as pd
import networkx as nx
import torch


: 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

Using device: cpu



In [ ]:

dataset = r"C:\Users\vansh\AI and ML reading material\GraphGAN_Project\GraphGAN\bio-grid-human\bio-grid-human_train.txt"
df = pd.read_csv(dataset,
                sep = '\t',
                names = ["NodeIDfrom", "NodeIDto"],
                )
#create the graph networkx object from the above dataframe

G = nx.from_pandas_edgelist(df = df,
                             source = "NodeIDfrom",
                             target = "NodeIDto",
                             create_using=nx.Graph())
len(G)

9404

In [ ]:
import numpy as np

def read_embeddings(filename, n_node, n_embed):
    """read pretrained node embeddings
    """

    with open(filename, "r") as f:
        lines = f.readlines()[1:]  # skip the first line
        rng = np.random.default_rng(seed=42)
        embedding_matrix = rng.standard_normal(size = (n_node, n_embed))
        for line in lines:
            emd = line.split()
            embedding_matrix[int(emd[0]), :] = str_list_to_float(emd[1:])
        return embedding_matrix

def str_list_to_float(str_list):
    return [float(item) for item in str_list]
filename = r"C:\Users\vansh\AI and ML reading material\GraphGAN_Project\GraphGAN\test_embeddings.emb"
X = read_embeddings(filename=filename,
                                     n_node = 9436,
                                     n_embed = 50)
X[4095], X.shape 

(array([-0.350393  ,  0.27482653, -0.04657786,  0.34706479,  0.21028991,
         0.56745905,  0.21026784,  0.68328494, -0.66025382, -0.96301746,
        -0.40459678, -0.10537465,  0.4645738 , -0.20851436, -0.68482751,
         0.19438629, -0.16578251,  0.48305196, -0.51618749,  0.05385859,
        -0.60018659, -0.2697354 , -0.09805907,  0.55923522,  0.49758846,
         0.3781592 ,  0.36981776,  0.26204705,  0.33005163,  0.32088688,
         0.42399544,  0.38767719, -0.31314573,  0.30350986, -0.79478496,
        -0.25088778,  0.47500992, -0.05470566, -0.69956172,  0.37452087,
         0.07660769, -0.23233452, -0.05459533, -0.27948675,  1.32820129,
         0.04532199,  0.262227  , -0.01381476,  0.42898911,  0.42193207]),
 (9436, 50))

In [ ]:
rng = np.random.default_rng(seed = 42)

k=3
X = np.array(X)
node_embed_dim = 50
centroid = rng.standard_normal(size = (k, node_embed_dim))

In [ ]:

#K-Means Algorithm for node embeddings
#the embediddings are 1xd dimensional.
class KMeansNodeClustering:

  def __init__(self, X, k, node_embed_dim):
      self.k = k #k is the number of cluster centres
      self.X = X #dataset consisting of node embeddings
      self.node_embed_dim = node_embed_dim #dimensions of node embeddings.
      self.centroid = None #these will be the centres of our distribution

  @staticmethod
  def euclidean_measure(centroid, node_embed):  # calculates the distance of the k-dimensional node from the centre
      return np.sqrt(np.sum((centroid - node_embed)**2, axis=1))

  def dimensional_mean(cluster_num, node_embed_dim, cluster_indices, cluster_centres): #calculates the mean of of the arrays, dimension-wise
        axis_centre = np.zeros(node_embed_dim)
        for i in range(node_embed_dim):
            y = 0
            tup = cluster_indices[cluster_num].shape
            shape = tup[0]
            for x in range(shape):
                y += X[cluster_indices[cluster_num][x]][0][i]
            mean = np.mean(y)
            axis_centre[i] += mean
        axis_centre = axis_centre/np.linalg.norm(axis_centre)
        return cluster_centres.append(axis_centre)

  def fit(self, max_iterations = 200):
      rng = np.random.default_rng(seed = 69)
      self.centroid = rng.standard_normal(size = (self.k, self.node_embed_dim))
      self.centroid  = (self.centroid)/(np.max(self.centroid))
      for _ in range(max_iterations):
          y = []

          for node_embed in self.X:
                distance = KMeansNodeClustering.euclidean_measure(node_embed = np.array(node_embed),
                                                                  centroid = centroid)
                cluster_num = np.argmin(distance)
                y.append(cluster_num)

          y = np.array(y) #stores the clustur number each of the nodes belong to

          cluster_indices = [] #to know which node belongs to which cluster

          for i in range(self.k):
                cluster_indices.append(np.argwhere(y == i)) #returns every node which belongs to the same cluster
          cluster_indices = np.array(cluster_indices, dtype = object)
          cluster_centres = [] #stores the centres of the clusters

          for j, indices in enumerate(cluster_indices): #cluster_indices contains the cluster numbers and the indices that belong to a particular cluster
                #i = cluster number
                #indices = indices of the nodes that belong to i.
                if len(indices) == 0:
                    cluster_centres.append(self.centroid[i])
                else:
                    KMeansNodeClustering.dimensional_mean(
                                                          cluster_num = j,
                                                          node_embed_dim=node_embed_dim,
                                                          cluster_indices = cluster_indices,
                                                          cluster_centres=cluster_centres)

          self.centroid = np.array(cluster_centres)
          y = np.array(y)
      return y, cluster_indices




In [ ]:
clustered_nodes = KMeansNodeClustering(X = X,
                                       k = k,
                                       node_embed_dim = node_embed_dim)
#cluster_indices_id stores the number_id of the clusters each embeddings is related to
#cluster_indices stores all the nodes that belong to one particular cluster, for all the clusters.
cluster_indices_id, cluster_indices = clustered_nodes.fit()


In [ ]:
centroid_array = clustered_nodes.centroid #contains the centroids of all the clusteres.
centroid_array

array([[ 0.02405422, -0.16886884,  0.0432893 , -0.02474142, -0.01689123,
         0.02418717, -0.00320072,  0.31513042, -0.06785535, -0.02390776,
        -0.01088693, -0.12602707,  0.09837792,  0.25126294, -0.14625928,
         0.18393429,  0.0771853 ,  0.0515889 , -0.39699451, -0.01127475,
        -0.06527127,  0.0611539 ,  0.08479745, -0.08870684, -0.03389312,
         0.10036672, -0.0599632 ,  0.12631391, -0.09282915,  0.02503458,
         0.24897943,  0.01432958, -0.29952319,  0.05440489,  0.02150592,
         0.10075214,  0.19389914, -0.15815777, -0.17780445, -0.10890294,
         0.19445924, -0.0771647 ,  0.01710894,  0.12998771,  0.29910963,
        -0.0868231 ,  0.20431098, -0.08287313, -0.06864406,  0.08207669],
       [ 0.01025421,  0.34934564, -0.04813136, -0.17420928,  0.13873117,
        -0.00400628,  0.08470334,  0.41181313, -0.03107169,  0.11612725,
        -0.11671086, -0.0533821 , -0.11453404, -0.09141086, -0.04753912,
         0.06836953,  0.02549791,  0.17110002, -0.

In [ ]:
cluster_indices = np.array(cluster_indices, dtype=object)
cluster_indices_array = np.array([torch.tensor(np.squeeze(cluster_indices[0], axis = 1)),
                                  torch.tensor(np.squeeze(cluster_indices[1], axis = 1)),
                                  torch.tensor(np.squeeze(cluster_indices[2], axis = 1)),
                                  ], dtype = object)


cluster_indices_array

C:\Users\vansh\AppData\Local\Temp\ipykernel_39776\1584236110.py:2: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  cluster_indices_array = np.array([torch.tensor(np.squeeze(cluster_indices[0], axis = 1)),


array([tensor([   0,    1,    3,  ..., 9427, 9429, 9432]),
       tensor([   2,    9,   12,  ..., 9433, 9434, 9435]),
       tensor([  30,   47,   73,  105,  132,  264,  306,  350,  401,  425,  448,  474,
                552,  707,  709,  740, 1000, 1027, 1049, 1062, 1108, 1183, 1202, 1203,
               1205, 1460, 1559, 1561, 1605, 1606, 1642, 1663, 1678, 1723, 1733, 1739,
               1740, 1754, 1876, 2155, 2158, 2177, 2210, 2221, 2237, 2254, 2298, 2300,
               2316, 2347, 2686, 2719, 2739, 2769, 2811, 2907, 2928, 2978, 2996, 3011,
               3138, 3263, 3346, 3369, 3464, 3476, 3480, 3481, 3719, 3809, 3842, 3863,
               3872, 3887, 3940, 3946, 4043, 4049, 4221, 4312, 4327, 4453, 4479, 4538,
               4603, 4707, 4719, 4730, 4795, 4805, 4819, 4855, 5008, 5030, 5137, 5258,
               5271, 5418, 5526, 5774, 5874, 5887, 5908, 5957, 6174, 6175, 6183, 6248,
               6549, 6608, 6630, 6758, 6771, 6851, 6908, 6998, 7134, 7243, 7251, 7516,
            

In [ ]:
import torch

class FuzzyLayer:
    """
    This is the parent class which will contain variables for Antecedant, Inference and Consequent classes.

    Args:
        X: The dataset which contains all the embeddings. shape(no_of_nodes, node_embed_dim)
        centroid_array: contains all the centroids of all the clusters. shape(no_of_clusters, node_embed_dim)
        cluster_indices_array: contains all the indices that belong to one particular cluster. shape(no_of_clusters, *number_of_indices_per_cluster*)
                                                                                               number_of_indices_per_cluster varies therefore it has no particular shape
    """

    def __init__(self, X, centroid_array, cluster_indices_array):
        self.X = torch.tensor(X, device=device, dtype=torch.float64)
        self.centroid_array = torch.tensor(centroid_array, device=device, dtype=torch.float64)
        self.cluster_indices_array = cluster_indices_array

class Antecedant(FuzzyLayer):
    """
    Antecedant part of the fuzzy logic system, it gives us membership functions for the node embeddings.
    """
    def __init__(self, X, centroid_array, cluster_indices_array):
        super().__init__(X, centroid_array, cluster_indices_array)

    def dot_product(self):
        # Use matrix multiplication instead of nested loops for efficiency
        dot_products = torch.matmul(self.X, self.centroid_array.T)
        return dot_products

    def get_points(self, cluster_num, embed_axis):
        # Fetch points corresponding to the cluster_num and embed_axis
        cluster_ids = torch.tensor(self.cluster_indices_array[cluster_num], device=device)
        points = self.X[cluster_ids, embed_axis]
        return points

    def get_mean(self, cluster_num, embed_axis):
        # Directly access the mean from centroid array
        return self.centroid_array[cluster_num, embed_axis]

    def get_standard_deviation(self, cluster_num, embed_axis):
        # Compute the variance in a vectorized manner
        points = self.get_points(cluster_num, embed_axis)
        mean = self.get_mean(cluster_num, embed_axis)
        variance = torch.var(points)
        variance = torch.sqrt(variance)
        return variance

    def get_stddev_tensor(self, embed_axis):
        # Vectorize variance computation for all clusters
        stddev = torch.tensor([self.get_standard_deviation(cluster_num, embed_axis) for cluster_num in range(self.centroid_array.shape[0])], device=device)
        return stddev

    def gaussianMF(self, cluster_num, embed_axis, element):
        # Vectorized Gaussian Membership Function calculation
        mean = self.get_mean(cluster_num, embed_axis)
        stddev_tensor = self.get_stddev_tensor(embed_axis)
        if stddev_tensor[cluster_num] != 0:
            gaussian = (element - mean) ** 2 / (2 * (stddev_tensor[cluster_num] ** 2))
        else:
            return 0.0
        return torch.exp(-gaussian)

    def get_membership_array(self, embed_axis):
        # Preallocate the membership array
        membership_array = torch.zeros((self.X.shape[0], self.centroid_array.shape[0]), device=device)
        for node_embed_num in range(self.X.shape[0]):
            for i in range(self.centroid_array.shape[0]):
                membership = self.gaussianMF(cluster_num=i, embed_axis=embed_axis, element=self.X[node_embed_num, embed_axis])
                membership_array[node_embed_num, i] = membership
        return membership_array



In [ ]:
b = Antecedant(X = X,
               centroid_array = centroid_array,
               cluster_indices_array = cluster_indices_array)


In [ ]:
membership_array = []
for embed_axis in range(node_embed_dim):
    mem_array_per_axis = b.get_membership_array(embed_axis = embed_axis)
    membership_array.append(mem_array_per_axis)
membership_array = torch.stack(membership_array)
membership_array

C:\Users\vansh\AppData\Local\Temp\ipykernel_39776\4202830216.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cluster_ids = torch.tensor(self.cluster_indices_array[cluster_num], device=device)


tensor([[[4.9446e-01, 4.0103e-01, 9.9384e-01],
         [1.4409e-03, 2.5523e-04, 9.4634e-01],
         [9.8327e-01, 9.8399e-01, 9.9991e-01],
         ...,
         [8.2893e-01, 8.0486e-01, 9.9863e-01],
         [8.2538e-01, 8.0072e-01, 9.9860e-01],
         [5.4269e-01, 4.8098e-01, 9.9529e-01]],

        [[6.1268e-01, 8.5506e-01, 9.9726e-01],
         [9.3263e-02, 1.7161e-02, 9.7155e-01],
         [9.5033e-01, 9.8689e-01, 9.9997e-01],
         ...,
         [2.5932e-01, 4.5053e-01, 9.9039e-01],
         [1.4871e-01, 2.7989e-01, 9.8571e-01],
         [2.9414e-01, 4.9953e-01, 9.9144e-01]],

        [[2.2588e-02, 3.4584e-02, 9.3537e-01],
         [1.8767e-01, 2.0313e-01, 9.6994e-01],
         [5.3075e-01, 5.0453e-01, 9.8777e-01],
         ...,
         [6.0163e-01, 7.8424e-01, 9.9322e-01],
         [5.3058e-01, 7.2601e-01, 9.9136e-01],
         [9.9890e-01, 9.7479e-01, 9.9983e-01]],

        ...,

        [[6.8289e-05, 5.5139e-17, 8.4344e-01],
         [2.7363e-01, 1.5365e-02, 9.7178e-01]

In [ ]:
import torch
import numpy as np  # Only if absolutely necessary

class Consequent(FuzzyLayer):
    """
    This class generates the crisp embeddings using the membership values generated by the Antecedant class.

    Essentially, we fuzzified each of the embeddings for each node, clustered the nodes, found centroids,
    and calculated membership for each embedding. Now, we use that membership to output crisp memberships.

    To do that, we generate sets with variances and treat them as the consequent fuzzy set from which crisp output is generated.
    """

    def __init__(self, X, centroid_array, cluster_indices_array):
        super().__init__(X, centroid_array, cluster_indices_array)

    def get_average_membership_array(self, membership_array):
        """
        Calculates the average membership for each embedding dimension across all clusters.

        Args:
            membership_array (torch.Tensor): The membership values of shape (node_embed_dim, num_nodes, num_clusters).

        Returns:
            torch.Tensor: The average membership values of shape (node_embed_dim, num_nodes).
        """
        return torch.mean(membership_array, dim=2)  # shape = (node_embed_dim, num_nodes)

    def get_embed_axis_mean(self):
        """
        Calculates the mean of each embedding axis.

        Returns:
            torch.Tensor: The mean values for each embedding axis of shape (node_embed_dim,).
        """
        a = self.X.transpose(dim0=1, dim1=0) #shape = (50, 5242)
        return np.mean(a, axis = 1) #shape = (50,)

    def get_embed_axis_variance(self):
        """
        Calculates the variance of each embedding axis.

        Returns:
            torch.Tensor: The variance values for each embedding axis of shape (node_embed_dim,).
        """
        a = self.X.transpose(dim0=1, dim1=0)
        mean = self.get_embed_axis_mean()
        variance = np.zeros(shape = node_embed_dim)
        for i in range(node_embed_dim):
            x = np.sqrt(np.mean((a[i] - mean[i])**2))
            variance[i] += x
        return variance #shape = (50,)

    def get_crisp_embeddings(self, average_membership_array):
        """
        Generates the crisp embeddings using the average membership values.

        Args:
            average_membership_array (torch.Tensor): The average membership values of shape (node_embed_dim, num_nodes).

        Returns:
            torch.Tensor: The crisp embeddings of shape (num_nodes, node_embed_dim).
        """
        a = self.X.transpose(dim0=1, dim1=0)
        mean = self.get_embed_axis_mean()
        variance = self.get_embed_axis_variance()
        new_X = np.zeros(shape = (node_embed_dim, self.X.shape[0]))
        for i in range(node_embed_dim):
            for j in range(self.X.shape[0]):
                if average_membership_array[i][j] == 0:
                    new_X[i][j] = a[i][j]
                else:
                    b = -1*np.log(average_membership_array[i][j])
                    if(a[i][j] < 0):
                        b = -1*np.sqrt(b)
                    else:
                        b = np.sqrt(b)
                    b = b*variance[i] + mean[i]
                    new_X[i][j] = b
        new_X = torch.transpose(new_X, dim0=1, dim1=0)
        new_X.shape
        #new_X = new_X/np.linalg.norm(new_X, axis = 0)
        return new_X

In [ ]:
c = Consequent(X = X,
               centroid_array=centroid_array,
               cluster_indices_array = cluster_indices_array)

In [ ]:
average_membership = c.get_average_membership_array(membership_array = membership_array)
average_membership 



tensor([[0.6298, 0.3160, 0.9891,  ..., 0.8775, 0.8749, 0.6730],
        [0.8217, 0.3607, 0.9791,  ..., 0.5667, 0.4714, 0.5950],
        [0.3308, 0.4536, 0.6744,  ..., 0.7930, 0.7493, 0.9912],
        ...,
        [0.2812, 0.4203, 0.3875,  ..., 0.9438, 0.7688, 0.9902],
        [0.9402, 0.9017, 0.9053,  ..., 0.9889, 0.9709, 0.9901],
        [0.2583, 0.2283, 0.6710,  ..., 0.9618, 0.9725, 0.8198]])

In [ ]:
crisp = c.get_crisp_embeddings(average_membership_array = average_membership)
norm = np.linalg.norm(crisp, axis = 1)
print(norm.shape)
for i in range(crisp.shape[0]):
    crisp[i] = crisp[i]
crisp


TypeError: mean() received an invalid combination of arguments - got (axis=int, dtype=NoneType, out=NoneType, ), but expected one of:
 * (*, torch.dtype dtype)
 * (tuple of ints dim, bool keepdim, *, torch.dtype dtype)
 * (tuple of names dim, bool keepdim, *, torch.dtype dtype)


In [ ]:
crisp[torch.tensor(np.array(G.nodes()))]

tensor([[ 0.8961, -0.8521, -0.4684,  ..., -1.2175,  0.8649,  0.0586],
        [-0.8627, -1.0469, -0.7183,  ...,  0.5976,  1.0827, -1.0759],
        [ 0.9045,  0.8788, -0.8165,  ..., -0.3847,  0.8627, -0.6265],
        ...,
        [ 0.0794,  1.2068, -0.3584,  ..., -0.3674, -0.1348, -0.4478],
        [-0.2495,  0.7903, -0.3105,  ..., -0.1242, -0.1881, -0.4330],
        [-0.1506,  0.9469, -0.2369,  ..., -0.6082, -0.0816, -0.6880]],
       dtype=torch.float64)

In [ ]:
import os
embedding_filename = r"C:\Users\vansh\AI and ML reading material\GraphGAN_Project\Fuzzy Representation Learning\abc.txt"
embeddings = crisp[torch.tensor(np.array(G.nodes()))]
index = np.array(G.nodes()).reshape(-1, 1)
embedding_matrix = np.hstack([index, embeddings])
embedding_list = embedding_matrix.tolist()
embedding_str = [str(int(emb[0])) + " " + " ".join([str(x) for x in emb[1:]]) + "\n"
                  for emb in embedding_list]
with open(embedding_filename, "w+") as f:
    lines = [str(G.number_of_nodes()) + "\t" + str(node_embed_dim) + "\n"] + embedding_str
    f.writelines(lines)
